# Lab 7: Linear Models, Feature Engineering and Logistic Regression

## Set up

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(42)
plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline

## Load the Tips Dataset

In [2]:
data = sns.load_dataset("tips")
print("Number of Records:", len(data))
data.head()

Number of Records: 244


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# Part 1: Defining the Model and Feature Engineering
Our prediction for tip be a combination of the following features:

$$
\text{Tip} = \theta_1 * \text{total_bill} + \theta_2 * \text{sex} + \theta_3 * \text{smoker} + \theta_4 * \text{day} + \theta_5 * \text{time} + \theta_6 * \text{size}
$$

Notice that some of these features are not numbers! But our linear model will need to predict a numerical value. 
### Task 1.1 Split the data into Y(tip) and X(all others)

In [3]:
tips = data['tip']
X = data.drop('tip', axis = 1)

## Task 1.2: Feature Engineering
First, let's convert everything to numerical values. A straightforward approach is to map some of these non-numerical features into numerical ones. For example, we can treat the day as a value from 1-7. However, one of the disadvantages in directly translating to a numeric value is that we unintentially assign certain features disproportionate weight. Consider assigning Sunday to the numeric value of 7. Monday is assigned to 1 and thus Sunday has 7 times the influence of Monday in our linear model which can lower the accuracy of our model.

solution: one-hot encoding! 

In [4]:
def one_hot_encode(data):
    return pd.get_dummies(X)

In [5]:
one_hot_X = one_hot_encode(X)
one_hot_X.head()

,total_bill,size,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,16.99,2,0,1,0,1,0,0,0,1,0,1
1,10.34,3,1,0,0,1,0,0,0,1,0,1
2,21.01,3,1,0,0,1,0,0,0,1,0,1
3,23.68,2,1,0,0,1,0,0,0,1,0,1
4,24.59,4,0,1,0,1,0,0,0,1,0,1


## Task 1.2: Defining the Model

In [6]:
def linear_model(thetas, X):
    """
    Return the linear combination of thetas and features as defined above.
    """
    return np.dot(X,thetas)

# Part 2: Fitting the Model: Numeric Methods

In [7]:
from scipy.optimize import minimize

def abserror(y, y_hat):
    return np.abs(y-y_hat)

def sqerror(y, y_hat):
    return (y - y_hat)**2

def minimize_average_loss(loss_function, model, x, y):
    """
    loss_function: either the squared or absolute loss functions from above.
    model: the model (as defined above)
    x: the x values (one-hot encoded data)
    y: the y values (tip amounts)
    
    return the estimate for each theta as a vector
    
    Note we will ignore failed convergence for this lab ... 
    """
    return minimize(lambda theta: sqerror(linear_model(theta,x),y).mean(), x0=np.array([0]*x.shape[1])).x


minimize_average_loss(sqerror, linear_model, one_hot_X, tips)

array([0.094487  , 0.1759912 , 0.18411073, 0.21655221, 0.15712765,
       0.24353564, 0.0151997 , 0.17746083, 0.05600925, 0.15199322,
       0.23440138, 0.16626186])

# Part 3: Fitting the Model: Analytic Methods
Let's also fit our model analytically, for the l2 loss function. In this question we will derive an analytical solution, fit our model and compare our results with our numerical optimization results.

## Task 3.1: Least Squares Solution
If we're fitting a linear model with the l2 loss function, we are performing least squares! 
$$\min_{\theta} ||X\theta - y||^2$$

Let's begin by deriving the analytic solution to least squares. Assume X is full column rank.

### BEGIN SOLUTION
Take the gradient and set theta equal to 0. 

$$2(X\theta - y) = 0$$
$$ (\theta^TX^T - y^T)(X\theta - y)$$
$$ \theta^TX^TX\theta - \theta^TX^Ty - y^TX\theta + y^Ty$$
$$ (X^TX + (X^TX)^T)\theta - 2X^Ty + \theta(y^Ty) = 0$$
$$ (X^TX+(X^TX)^T)\theta - 2X^Ty+0 = 0$$
$$ 2X^TX\theta = 2X^Ty$$
$$ \theta = (X^TX)^{-1}{X^Ty}$$
### END SOLUTION

## Task 3.2: Solving for Theta

In [8]:
def get_analytical(x, y):
    """
    x: our one-hot encoded dataset
    y: tip amounts
    """   
    return np.dot(np.dot(np.linalg.pinv(np.dot(np.transpose(x),x)), np.transpose(x)),y)

In [9]:
analytical_thetas = get_analytical(one_hot_X.astype(float), tips.astype(float))
print("Our analytical loss is: ", sqerror(linear_model(analytical_thetas, one_hot_X),tips).mean())
print("Our numerical loss is: ", sqerror(linear_model(minimize_average_loss(sqerror, linear_model, one_hot_X, tips), one_hot_X), tips).mean())

Our analytical loss is:  1.010353561225785
Our numerical loss is:  1.010353561236632


## Task 3.3: Weird Results?
Our analytical loss is surprisingly much worse than our numerical loss. Why is this? 

Hint: https://stackoverflow.com/questions/31256252/why-does-numpy-linalg-solve-offer-more-precise-matrix-inversions-than-numpy-li

# Part 4 - Logistic Regression
In this part we will implement a logistic regression model with the tip data set. We will use Male and Female as the binary classification. We will use the LogisiticRegression model from sklearn to find a prediction model.

## task 4.1
** create the X (one-hot features) and y (sex) **

In [10]:
X = one_hot_X.drop(['sex_Male','sex_Female'], axis = 1)
y = data['sex']

print(X.head())
print(y.head())


   total_bill  size  smoker_Yes  smoker_No  day_Thur  day_Fri  day_Sat  \
0       16.99     2           0          1         0        0        0   
1       10.34     3           0          1         0        0        0   
2       21.01     3           0          1         0        0        0   
3       23.68     2           0          1         0        0        0   
4       24.59     4           0          1         0        0        0   

   day_Sun  time_Lunch  time_Dinner  
0        1           0            1  
1        1           0            1  
2        1           0            1  
3        1           0            1  
4        1           0            1  
0    Female
1      Male
2      Male
3      Male
4    Female
Name: sex, dtype: category
Categories (2, object): [Male, Female]


## task 4.2 
Split the data set to training and test using sklearn train_test_split

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state=0)

print (X_train.head(), X_test.head(), y_train.head(), y_test.head())

     total_bill  size  smoker_Yes  smoker_No  day_Thur  day_Fri  day_Sat  \
7         26.88     4           0          1         0        0        0   
83        32.68     2           1          0         1        0        0   
176       17.89     2           1          0         0        0        0   
106       20.49     2           1          0         0        0        1   
156       48.17     6           0          1         0        0        0   

     day_Sun  time_Lunch  time_Dinner  
7          1           0            1  
83         0           1            0  
176        1           0            1  
106        0           0            1  
156        1           0            1        total_bill  size  smoker_Yes  smoker_No  day_Thur  day_Fri  day_Sat  \
64        17.59     3           0          1         0        0        1   
63        18.29     4           1          0         0        0        1   
55        19.49     2           0          1         0        0        0   

## task 4.3 logistic regression model
Fit a logisitic regression model using LogisticRegression from sklearn

In [12]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression().fit(X_train, y_train)
logmodel

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

## task 4.4 Predicting

In [13]:
predictions = logmodel.predict(X_test)

## task 4.5 reporting

In [14]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions,labels=['Male', 'Female']))


              precision    recall  f1-score   support

        Male       0.50      0.80      0.62        25
      Female       0.44      0.17      0.24        24

    accuracy                           0.49        49
   macro avg       0.47      0.48      0.43        49
weighted avg       0.47      0.49      0.43        49



Developed by A.D. Gunawardena. Credits: Josh Hug, Berkeley Data Science Lab